# PSET 9
### Juan M Jimenez R.

## Question 2
### (a)

In [1]:
using Distributions, Random, PrettyTables, QuadGK, Plots, Roots, JLD2, Optim, Plots

In [2]:
#Generating data proccess
function genData(n,t,a,b,g,d,m)
    
    #Creating vars 
    v0=randn(n,1)
    v1=randn(n,1)
    x2=rand(Normal(m, 1), n)
    x1=ones(n,1)
    x=[x1 x2]
    
    #Probit vars
    t0=t[1]
    t1=t[2]
    e=randn(n,1)
    xt=t0 .+ t1*x2 
    T=(xt .> e)
    ps=cdf.(Normal(0,1), xt)
    
    #True regression model for Yi
    Δ= g-a + (d-b)*mean(ps)
    u= v0 .+ T.*(v1-v0)
    y= a .+ b.*ps+ Δ.*T+ (d-b).*(ps .-mean(ps)).*T .+ u

    return x, T, y
end

genData (generic function with 1 method)

### (b)

In [3]:
#Probit function 
Φ(v)=cdf(Normal(0,1),v)
ϕ(v)=pdf(Normal(0,1),v)

function mleProbit(t, x, d)
    n=length(d)
    L=0
    x=x[:,2]
    
    for i=1:n
        xt= t[1] .+ x[i]*t[2]
        L -= d[i]*log(Φ(xt)) + (1-d[i])*log(1-Φ(xt))
    end
    
    #Returning Log likelihood summ 
    return L/n
end

mleProbit (generic function with 1 method)

In [4]:
#ATE estimator function 
function ateMatch(x, d, y)
    
    #First stage - Probit estimation
    result=optimize(t->mleProbit(t,x,d),[0.0;0.0;])
    τ=Optim.minimizer(result)
    
    #Creating second stage vars 
    x2=x[:,2]
    xt= τ[1] .+ x2*τ[2]
    z1= ones(length(d))
    z2= Φ(xt)
    z3= d
    z4= (Φ(xt) .- mean(Φ(xt))).*D
    z=[z1 z2 z3 z4]
    
    #OLS estimation
    θ=(z'*z)\(z'*y)
    
    #OLS variance
    B= (z'*z)/n
    u= y - z*θ
    zu= z.*u
    Ω= (zu' * zu)/n
    var1= (B\Ω/B)/n
    σ1= sqrt(var1)
    
    #OLS corrected variance 
    pdfxt=pdf.(Normal(0,1), xt)
    cdfxt=cdf.(Normal(0,1), xt)
    
    s=(d.*(pdfxt./cdfxt) - (1 .-d).*(pdfxt./(1 .-cdfxt))).*x
    C=(s'*s)/n
    
    #=PAUL:
    #First-step correction
    #X=[ones(n,1) x]
    f=pdfxt
    f2=repeat(f,1,2)
    fX=f2.*x
    mfX=mean(fX,dims=1)
    dZ=θ[2]*fX+θ[4]*(fX-repeat(mfX,n,1))
    Λ=z'*dZ/n
    
    #Std.err of Δ̂
    #B̂inv*(Ω̂+Λ̂*Ĉinv*Λ̂')*B̂inv
    AsyVar=(B\(Ω + Λ*inv(C)*Λ')/B)/n 
    σ2= sqrt(AsyVar)
    =#
    
    #MY WAY:
    h=zeros(length(d),2)
    
    for i= 1:length(d)
       h[i,:]=θ[2]*pdfxt[i]*x[i,:]' + θ[4]*(pdfxt[i]*x[i,:]' .- mean(pdfxt[i]*x[i,:]'))
    end 
    
    Λ=(z'*h)/n
    var2= (B\(Ω + Λ*inv(C)*Λ')/B)/n
    σ2= sqrt(var2)
    
    return τ, θ, σ1, σ2    
end  

#Notes: here estimated pscore is Φ(xt)
#Why extra 1/n in var1?

ateMatch (generic function with 1 method)

### (c)

In [5]:
function ateBoots(x, d, y, Δ, B, η)
    n=length(d)
    Δb = zeros(B)
    
    for i= 1:B
        # Select an n random sample with replacement (from our original sample)
        idn = sample(1:n, n, replace = true)
        yb = y[idn]
        xb = x[idn, :]
        db = d[idn]

        #Estimating the ATE in each B   
        t, b, se1, se2 = ateMatch(xb, db, yb);
        Δb[i]= b[3]       
    end
    
    #Bootstrap SE
    σb=sqrt(mean((Δb .- Δ).^2))
    
    #Bootstrap CIs
    Δbₛ= sort(Δb)
    ci_lb= Δbₛ[Int(η/2*(B+1))]
    ci_ub= Δbₛ[Int((1-η/2)*(B+1))]
    
    return σb, ci_lb, ci_ub
    
end

ateBoots (generic function with 1 method)

### (d)

In [6]:
Random.seed!(42);

#Defining true parameters
n=100
τ₀₀=-.5
τ₀₁=2
τ₀=[τ₀₀ τ₀₁]'
α₀=1
β₀=1
γ₀=1
δ₀=1
μ₀=1

B=999
R=1000
η=.1

zstat= quantile(Normal(0,1), 1-(η/2))

#Calculating real ATE
ate= γ₀-α₀ + (δ₀-β₀)*(quadgk(z->Φ(τ₀₀+τ₀₁*z)*ϕ(z),-Inf,Inf,atol=1e-8)[1])

0.0

In [7]:
tinci_ols = 0
tinci_ols_2stp = 0
tinci_ols_boots = 0
tinci_ols_pboots = 0

#Why hve to run it before to work?
X, D, Y=genData(n, τ₀, α₀, β₀, γ₀, δ₀, μ₀);
t, b, se1, se2 = ateMatch(X, D, Y);

#Simulations
for r=1:R
    #GDP
    X, D, Y=genData(n, τ₀, α₀, β₀, γ₀, δ₀, μ₀);
    
    #2-step estimation
    t, b, se1, se2 = ateMatch(X, D, Y);
    Δ= b[3]   
    σ_ols=se1[3,3]
    σ_ols_2stp=se2[3,3]
    
    #Bootstrap estimation
    σ_boots, ci_lb, ci_ub = ateBoots(X, D, Y, Δ, B, η)
        
    #Calculating absolute coverages (times that real ATE is inside CIs)
    tinci_ols += (abs(ate-Δ)<=zstat*σ_ols)
    tinci_ols_2stp += (abs(ate-Δ)<=zstat*σ_ols_2stp)  
    tinci_ols_boots += (abs(ate-Δ)<=zstat*σ_boots)  
    tinci_ols_pboots += (ate>=ci_lb)*(ate<=ci_ub)
end

#Calculating mean coverages
minci_ols= tinci_ols/R
minci_ols_2stp= tinci_ols_2stp/R
minci_ols_boots= tinci_ols_boots/R 
minci_ols_pboots= tinci_ols_pboots/R

0.878

In [8]:
#Table report
table_data = ["ate (estimate)" b[3] b[3] b[3] b[3];
        "Coverage Prob of CI" minci_ols minci_ols_2stp minci_ols_boots minci_ols_pboots;]
header=["Statistic", "No correction", "2 step correction", "SE bootstrap", "Percentile Bootstrap"]
pretty_table(table_data; header)

┌─────────────────────┬───────────────┬───────────────────┬──────────────┬──────────────────────┐
│           Statistic │ No correction │ 2 step correction │ SE bootstrap │ Percentile Bootstrap │
├─────────────────────┼───────────────┼───────────────────┼──────────────┼──────────────────────┤
│      ate (estimate) │      0.017278 │          0.017278 │     0.017278 │             0.017278 │
│ Coverage Prob of CI │         0.755 │             0.761 │        0.893 │                0.878 │
└─────────────────────┴───────────────┴───────────────────┴──────────────┴──────────────────────┘


### (e)

It seems that bootstrap is most accurate to construct the confidence intervals (either the standard error or the percentile method). This since its coverages are really close to the real nominal probability of 90%.

Regarding the correction of the two step stimator, it seems it helps to improve the coverage but the improvement seems marginal compared to the bootstrap method. 